In [77]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Flatten,Attention, Concatenate, Input ,Reshape ,MultiHeadAttention
from tensorflow.keras.optimizers import Adam,RMSprop
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error,mean_absolute_error,precision_score
import numpy as np
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend as K



In [78]:
file_path = r"/Users/I578087/Desktop/WaterLevel-main/data_2/dataset.csv"
data = pd.read_csv(file_path)

In [79]:
data.drop(['Feature9'],axis=1,inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Feature1  300 non-null    float64
 1   Feature2  300 non-null    float64
 2   Feature3  300 non-null    float64
 3   Feature4  300 non-null    float64
 4   Feature5  300 non-null    int64  
 5   Feature6  300 non-null    int64  
 6   Feature7  300 non-null    float64
 7   Feature8  300 non-null    float64
dtypes: float64(6), int64(2)
memory usage: 18.9 KB


In [80]:
# Initialize the StandardScaler
scaler = StandardScaler()

# Standardize the data
data = scaler.fit_transform(data)

In [81]:
# Create sequences for training (8 rows per year)
def create_sequences(data, seq_length=8, pred_length=20):
    X, y = [], []
    # Ensure the data is in numpy array format
    data = np.array(data)
    
    for i in range(len(data) - seq_length - pred_length + 1):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length:i+seq_length+pred_length])  # Predict the next pred_length steps
        
    return np.array(X), np.array(y)

# Generate sequences
X, y = create_sequences(data, seq_length=8, pred_length=20)

# Print shapes
print(X.shape)
print(y.shape)

(273, 8, 8)
(273, 20, 8)


In [82]:
X, y = create_sequences(data)

In [83]:
# Split the dataset into training and testing sets (use last 20 rows for testing)
X_train = X[:-20]
y_train = y[:-20]
X_test = X[-20:]
y_test = y[-20:]

In [84]:
import keras.backend as K
import tensorflow as tf
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention

@tf.keras.utils.register_keras_serializable()
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

@tf.keras.utils.register_keras_serializable()
def mape(y_true, y_pred):
    return K.mean(K.abs((y_true - y_pred) / K.clip(K.abs(y_true), K.epsilon(), None))) * 100

@tf.keras.utils.register_keras_serializable()
def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))

@tf.keras.utils.register_keras_serializable()
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [85]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention

# Define custom objects
custom_objects = {
    'LayerNormalization': LayerNormalization,
    'MultiHeadAttention': MultiHeadAttention,
    'rmse': rmse,
    'mape': mape,
    'accuracy': accuracy,
    'precision': precision
}

# Load the model with custom objects
model1 = load_model(r'/Users/I578087/Desktop/WaterLevel-main/Models/Test/RNN_test.keras', custom_objects=custom_objects)

/Users/I578087/Desktop/WaterLevel-main/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 22 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [86]:
from tensorflow.keras.models import load_model
model2 = load_model(r'/Users/I578087/Desktop/WaterLevel-main/Models/Test/LSTM_test.h5')

In [87]:
#Make predictions
y_pred1 = model1.predict(X_test)

# Print predictions
print("Predictions: ", y_pred1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
Predictions:  [[[-1.4668242  -1.5068566  -1.5583725  ... -1.0436114   1.238088
   -1.4012194 ]
  [-1.4977429  -1.5695765  -1.5366007  ... -0.9657346   1.3785352
   -1.4264958 ]
  [-1.5330777  -1.5577582  -1.6020103  ... -0.9993347   1.4941614
   -1.5514101 ]
  ...
  [-1.2171576  -1.2740614  -1.2720159  ... -0.90501547  0.9753388
   -1.2026341 ]
  [-1.263959   -1.2134795  -1.3233235  ... -1.0152754   0.94734895
   -1.1410606 ]
  [-1.218692   -1.2322642  -1.2121483  ... -0.9676005   0.9131253
   -1.1922312 ]]

 [[-1.4671286  -1.5063844  -1.556103   ... -1.0461738   1.2368788
   -1.3998781 ]
  [-1.4968276  -1.5694723  -1.5338957  ... -0.9684894   1.3753179
   -1.4253428 ]
  [-1.5297236  -1.5547131  -1.5983301  ... -1.0011445   1.4877023
   -1.5503016 ]
  ...
  [-1.2134919  -1.2683647  -1.2684076  ... -0.8999457   0.9736886
   -1.1992401 ]
  [-1.2613332  -1.2094451  -1.3170652  ... -1.0082002   0.94683623
   -1.137188  ]
  [-1.2137617  -1.2267977  -1.

In [88]:
#Make predictions
y_pred2 = model2.predict(X_test)

# Print predictions
print("Predictions: ", y_pred2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Predictions:  [[[-1.5075035  -1.5930716  -1.6001717  ... -1.1188248   1.3619933
   -1.4421372 ]
  [-1.5256069  -1.5537875  -1.632324   ... -1.0357428   1.4178135
   -1.3887614 ]
  [-1.5156928  -1.6076264  -1.525548   ... -1.060903    1.4458565
   -1.5052351 ]
  ...
  [-1.3593801  -1.3743854  -1.3821052  ... -0.9034008   1.3086367
   -1.3569803 ]
  [-1.4275851  -1.4679546  -1.3648156  ... -0.9460331   1.2236316
   -1.2849902 ]
  [-1.3713567  -1.437009   -1.3670635  ... -0.87898815  1.227767
   -1.3194577 ]]

 [[-1.502134   -1.5852344  -1.5910382  ... -1.112784    1.3586737
   -1.4361103 ]
  [-1.5205667  -1.548575   -1.6234928  ... -1.0305029   1.4107542
   -1.3814107 ]
  [-1.5095645  -1.6009109  -1.5191474  ... -1.0552015   1.4382645
   -1.4981288 ]
  ...
  [-1.3505225  -1.3630806  -1.3717716  ... -0.8961766   1.2974566
   -1.3446465 ]
  [-1.4150194  -1.4559762  -1.3517029  ... -0.9399293   1.2152613
   -1.2753857 ]
  [-1.3611832  -1.4251331  -1.35

In [89]:
# Example RMSE values
rmse_model1 = 0.40163204073905945
rmse_model2 = 0.38472655415534973

# Calculate inverse RMSE
inv_rmse_model1 = 1 / rmse_model1
inv_rmse_model2 = 1 / rmse_model2

# Calculate total inverse RMSE
total_inv_rmse = inv_rmse_model1 + inv_rmse_model2

# Calculate weights
weight1 = inv_rmse_model1 / total_inv_rmse
weight2 = inv_rmse_model2 / total_inv_rmse

print(f"Weight for model1: {weight1}")
print(f"Weight for model2: {weight2}")

Weight for model1: 0.4892507777663575
Weight for model2: 0.5107492222336425


In [90]:
# Weighted Averaging
y_pred_X = (weight1 * y_pred1 + weight2 * y_pred2) / (weight1 + weight2)

# Print predictions
print(y_pred_X)

[[[-1.487601   -1.5508908  -1.5797215  ... -1.0820266   1.3013725
   -1.4221182 ]
  [-1.5119743  -1.5615122  -1.5854913  ... -1.0014912   1.3985965
   -1.407223  ]
  [-1.5241984  -1.5832283  -1.5629572  ... -1.0307806   1.4694897
   -1.5278263 ]
  ...
  [-1.2897975  -1.3253018  -1.328244   ... -0.9041908   1.1455704
   -1.2814664 ]
  [-1.347531   -1.3434525  -1.3445156  ... -0.9799099   1.0884601
   -1.2145725 ]
  [-1.2966654  -1.3368374  -1.2912712  ... -0.9223418   1.0738282
   -1.257212  ]]

 [[-1.4850075  -1.546657   -1.5739461  ... -1.080195    1.2990854
   -1.4183836 ]
  [-1.5089524  -1.558799   -1.5796573  ... -1.0001627   1.3934169
   -1.4029045 ]
  [-1.5194274  -1.5783086  -1.5578876  ... -1.0287541   1.4624519
   -1.5236545 ]
  ...
  [-1.2834802  -1.3167408  -1.3212006  ... -0.8980206   1.1390529
   -1.2735063 ]
  [-1.3398283  -1.3353608  -1.3347564  ... -0.97333086  1.0839342
   -1.2077724 ]
  [-1.289057   -1.3280973  -1.281653   ... -0.91365016  1.0682971
   -1.2491162 ]]



In [91]:
# Calculate and print final evaluation metrics on the test set
final_rmse = np.sqrt(mean_squared_error(y_test.flatten(), y_pred_X.flatten()))
final_mae = mean_absolute_error(y_test.flatten(), y_pred_X.flatten())
final_mape = np.mean(np.abs((y_test.flatten() - y_pred_X.flatten()) / y_test.flatten())) * 100
final_precision = precision_score((y_test.flatten() > 0.5).astype(int), (y_pred_X.flatten() > 0.5).astype(int))

print("Final RMSE: ", final_rmse)
print("Final MAE: ", final_mae)
print("Final MAPE: ", final_mape)
print("Final Precision: ", final_precision)

Final RMSE:  0.36445731651774077
Final MAE:  0.31176446120447365
Final MAPE:  127.69303329328051
Final Precision:  0.805


In [92]:
# Inverse transform the predictions and the test data
y_pred_inverse = scaler.inverse_transform(y_pred_X.reshape(-1, y_pred_X.shape[-1])).reshape(y_pred_X.shape)
y_test_inverse = scaler.inverse_transform(y_test.reshape(-1, y_test.shape[-1])).reshape(y_test.shape)

columnsnames=['Feature1', 'Feature2', 'Feature3', 'Feature4', 'Feature5', 'Feature6','Feature7', 'Feature8']

## Convert predictions and actual values to separate DataFrames using original column names
actual_df = pd.DataFrame(y_test_inverse.reshape(-1, y_test_inverse.shape[-1]), columns=columnsnames)
predicted_df = pd.DataFrame(y_pred_inverse.reshape(-1, y_pred_inverse.shape[-1]), columns=columnsnames)


In [93]:
actual_df

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8
0,0.1208,86623.6320,84400.4660,1162.3136,422.0,1083.0,0.7723,0.1413
1,0.1208,86623.6320,84400.4660,1162.3136,422.0,1083.0,0.7723,0.1413
2,0.1208,86635.5211,84412.6135,1162.3136,422.0,1083.0,0.7722,0.1413
3,0.1293,89857.0463,87010.3655,1197.5529,442.0,1113.0,0.7758,0.1526
4,0.1297,90049.8254,86932.1158,1178.7947,439.0,1094.0,0.7706,0.1535
...,...,...,...,...,...,...,...,...
395,0.1387,92597.5043,90066.2099,1181.6302,449.0,1093.0,0.7577,0.1645
396,0.1386,92571.9423,90039.6155,1181.6302,449.0,1093.0,0.7579,0.1644
397,0.1386,92571.9423,90039.6155,1181.6302,449.0,1093.0,0.7579,0.1644
398,0.1387,92597.5043,90066.2099,1181.6302,449.0,1093.0,0.7577,0.1645


In [94]:
predicted_df

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8
0,0.122134,86766.703125,83976.804688,1165.999634,421.160919,1084.445923,0.774756,0.143673
1,0.121633,86697.804688,83945.117188,1166.680420,425.545593,1087.256958,0.776000,0.144102
2,0.121382,86556.945312,84068.859375,1167.063354,423.838409,1086.234619,0.776907,0.140626
3,0.122488,87019.968750,83762.273438,1167.207520,426.527283,1084.979736,0.778141,0.141843
4,0.121701,86721.914062,84147.882812,1169.439209,429.149750,1089.444702,0.778704,0.141760
...,...,...,...,...,...,...,...,...
395,0.128349,88765.062500,86245.343750,1181.656860,437.241943,1098.294678,0.771540,0.152163
396,0.128676,88942.046875,86282.195312,1177.154053,434.705017,1095.778076,0.770360,0.150860
397,0.127966,89235.609375,86188.531250,1175.480591,432.389099,1093.645386,0.770058,0.150280
398,0.127783,89027.226562,86180.109375,1173.588745,430.903046,1090.604736,0.769587,0.152374
